In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pickle
import time

In [2]:
debut_df = pd.read_csv("../data/debut_album_1118.csv")

In [3]:
debut_df.shape[0]

1949

In [4]:
df = debut_df[60:80]

In [5]:
def get_followers(dataframe):
    
    ls = list(dataframe['artist'])
    sns_df = pd.DataFrame(columns=['artist', 'twitter', 'instagram', 'facebook', 'spotify', 'soundcloud', 'youtube'])
    
    # load webdriver and target webpage
    driver = webdriver.Chrome()
    driver.get('https://chartmetric.io/login')
    time.sleep(10)
    
    # load password for the website
    pickle_pw = pickle.load(open("chartmetric_pw.pickle", "rb"))

    # login
    driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(4) > div > input" ).send_keys( "lucaseo1991@gmail.com" )
    driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(5) > div > input" ).send_keys( pickle_pw )
    driver.find_element_by_css_selector("body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(7) > div > button").click()
    time.sleep(7)

    
    
    #input artist
    for artist in ls:
        
        # browse to search page
        driver.get('https://chartmetric.io/search')
        time.sleep(10)

        # input artist name
        driver.find_element_by_css_selector( "body > div:nth-child(1) > div:nth-child(2) > div > div > form > input" ).send_keys(artist)
        driver.find_element_by_css_selector("body > div:nth-child(1) > div:nth-child(2) > div > div > form > input").send_keys(Keys.ENTER)
        time.sleep(10)

        # enter artist info page
#         keyword = driver.find_element_by_css_selector("#artist > ul > li > div.media-body > div.media-heading > a").text
#         driver.find_element_by_link_text(keyword).click()
        driver.find_element_by_xpath("//*[@id='artist']/ul/li/div[2]/div[1]/a").click()
            
        time.sleep(10)
        
        social = driver.find_element_by_id("socialStats")

        time.sleep(5)
        
        twitter = social.find_element_by_id("twitterfollowers-total-fans").text
        instagram = social.find_element_by_id("instagram-total-fans").text
        facebook = social.find_element_by_id("facebooklikes-total-fans").text
        spotify = social.find_element_by_id("spotify-total-fans").text
        soundcloud = social.find_element_by_id("soundcloud-total-fans").text
        youtube = social.find_element_by_id("youtubesubscribers-total-fans").text

        
        data = {
            "artist": artist, 
            "twitter":twitter, 
            "instagram":instagram, 
            "facebook":facebook,
            "spotify":spotify, 
            "soundcloud":soundcloud,
            "youtube":youtube,
        }    
        
        sns_df.loc[len(sns_df)] = data
        
        time.sleep(2)
       
    driver.close()
        
    return sns_df
        

In [6]:
df_1 = get_followers(df)

In [7]:
print(df_1.shape)
df_1.head()

(20, 7)


,artist,twitter,instagram,facebook,spotify,soundcloud,youtube
0,Cerebral Ballzy,"5,362","4,969","33,535","8,448","1,974","2,453"
1,Ford & Lopatin,"1,464",,,"3,386","6,930",
2,Phonte,"131,734",,"50,642","29,170",1,
3,Chiaki Kuriyama,,,,617,,
4,Scotty McCreery,"1,018,099","442,719","2,219,790","304,445",,"317,770"


In [8]:
df_1.to_csv('../data/sns/df_60-79.csv', index=False)